**Task 1: Implementation of a message passing GNN.**

This Colab notebook is the deliverable of task 1 containing the implementation and evaluation of the Message Passing layer (See CustomMPLayer). It is implemented as an instance of torch_geometric.nn.conv.MessagePassing, making it easily pluggable and compatible with Torch Geometric framework. Acknowldgement: The working of this layer is evaluated using PyTorch Lightning module, the skeleton/base code of which is obtained from the UvA DL tutorials: https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html and modified for appropriate use. Evaluation is done on the CORA dataset.

Usage: Simply run all the cells on Google Colab. The performance of the model (Training, Validation, and Testing accuracy) using the implementation of this custom Message Passing layer on CORA dataset will be calculated and displayed. The hyperparameters can be chosen as needed.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("device:", device)

device: cpu


In [ ]:
# Google Colab does not have torch-geometric installed by default. Hence, we do it here if necessary
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric

from torch_geometric.datasets import Planetoid
import torch_geometric.data as geom_data
import torch_geometric.loader as geom_loader
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.typing import Adj

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 19.9 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 34.8 MB/s 
     |████████████████████████████████| 407 kB 24.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=dc86ecd8f075d5dd8ac7124b94de138f6940d796c1c6766e7f3098782c1b61a4
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
# Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: 
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
# Loading the CORA dataset
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')

print("Dataset Name:", cora_dataset)
print("Number of graphs:", len(cora_dataset))
print("Number of classes:", cora_dataset.num_classes)
print("Number of Node features:", cora_dataset.num_node_features)
print("Dataset:", cora_dataset[0])
print("Is undirected?:", cora_dataset[0].is_undirected())
print("Has self loops?:", cora_dataset[0].has_self_loops())

Processing...
Done!


Dataset Name: Cora()
Number of graphs: 1
Number of classes: 7
Number of Node features: 1433
Dataset: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Is undirected?: True
Has self loops?: False


In [ ]:
# Task 1: Here is the implementation of the specified message passing operation

class CustomMPLayer(MessagePassing):
    """The implementation of the Message Passing operation as follows: 

    .. math::
        \mathbf{x}_i^{\prime} = \mathbf{W}ReLU \left(\mathbf{x}_i+\mathbf{W}_{msg}
        \sum_{j \in \mathcal{N}(i)} \left(\mathbf{x}_j log\left(\left| \mathbf{x}_j
        \right|+\epsilon\right)\right) \right),


    where :math:`\mathbf{x}_i` denotes the embedding of vertex i at input or 
    output, :math:`\mathcal{N}(i)` denotes the one-hop neighbourhood of vertex 
    i, :math:`\mathbf{W}` and :math:`\mathbf{W}_{msg}` are linear layers.

    with :math:`\hat{d}_i = 1 + \sum_{j \in \mathcal{N}(i)} e_{j,i}`, where
    :math:`e_{j,i}` denotes the edge weight from source node :obj:`j` to target
    node :obj:`i` (default: :obj:`1.0`)

    Args:
        in_channels (int): Size of each input sample, or :obj:`-1` to derive
            the size from the first input(s) to the forward method.
        out_channels (int): Size of each output sample.
        epsilon (float, optional): The value of the constant \epsilon as
            specified in the above formulation.       
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """
    def __init__(self, in_channels: int, out_channels: int, epsilon: int = 0.01):
        super().__init__(aggr='add')
        self.lin_msg = torch.nn.Linear(in_channels, in_channels)
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.epsilon = epsilon

    def forward(self, x: Tensor, edge_index: Adj) -> Tensor:
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # x = x * torch.log(torch.abs(x) + self.epsilon)
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j: Tensor) -> Tensor:
        # x_j has shape [E, in_channels]
        return x_j * torch.log(torch.abs(x_j) + self.epsilon)

    def update(self, aggr_out: Tensor, x: Tensor) -> Tensor:
        x = x + self.lin_msg(aggr_out)
        x = F.relu(x)
        x = self.lin(x)
        return x



In [ ]:
class GNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of "hidden" graph layers
            kwargs - Additional arguments for the graph layer
        """
        super().__init__()
        gnn_layer = CustomMPLayer
        print('yeah')

        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=in_channels,
                          out_channels=out_channels,
                          **kwargs)
            ]
            in_channels = c_hidden
        layers += [gnn_layer(in_channels=in_channels,
                             out_channels=c_out,
                             **kwargs)]
        self.layers = nn.ModuleList(layers)

    def forward(self, x, edge_index):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
        """
        for l in self.layers:
            # For graph layers, we need to add the "edge_index" tensor as additional input
            # All PyTorch Geometric graph layer inherit the class "MessagePassing", hence
            # we can simply check the class type.
            if isinstance(l, MessagePassing): 
                x = l(x, edge_index)
            else:
                x = l(x) # this is for if we want to use other layers like dropout etc.
        return x

In [ ]:
class NodeLevelGNN(pl.LightningModule):

    def __init__(self, **model_kwargs):
        super().__init__()

        self.model = GNNModel(**model_kwargs)
        self.loss_module = nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)

        # Only calculate the loss on the nodes corresponding to the mask
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            raise ValueError(f"Unknown forward mode: {mode}")

        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return loss, acc

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

In [ ]:
def train_node_classifier(dataset, **model_kwargs):
    pl.seed_everything(37)
    node_data_loader = geom_loader.DataLoader(dataset, batch_size=1) # TODO running on CPU for now
    trainer = pl.Trainer(gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=20)

    model = NodeLevelGNN(c_in=dataset.num_node_features, c_out=dataset.num_classes, 
                         **model_kwargs)
    trainer.fit(model, node_data_loader, node_data_loader)

    test_result = trainer.test(model, dataloaders=node_data_loader, verbose=False)
    batch = next(iter(node_data_loader))
    batch = batch.to(model.device)

    _, train_acc = model.forward(batch, mode="train")
    _, val_acc = model.forward(batch, mode="val")
    result = {"train": train_acc,
              "val": val_acc,
              "test": test_result[0]['test_acc']}
    return model, result

In [ ]:
# For printing the test scores
def print_results(result_dict):
    """
    To print test scores
    """
    if "train" in result_dict:
        print(f"Train accuracy: {(100.0*result_dict['train']):4.2f}%")
    if "val" in result_dict:
        print(f"Val accuracy:   {(100.0*result_dict['val']):4.2f}%")
    print(f"Test accuracy:  {(100.0*result_dict['test']):4.2f}%")

In [ ]:
node_class_model, node_class_results = train_node_classifier(dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2, 
                                                        epsilon=0.01)

print_results(node_class_results)

Global seed set to 37
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


yeah


Missing logger folder: /content/lightning_logs

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | GNNModel         | 2.1 M 
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.313     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2708. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1942: PossibleUserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Train accuracy: 99.29%
Val accuracy:   68.40%
Test accuracy:  71.10%
